### Turn DynaSent dataset to differnet formats.

SST-2 dataset seems to be too easy. We are switching to harder dataset requiring harder compositional skills to demonstrate DIITO.

In [26]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

In [ ]:
r1_dataset = load_dataset(
    "dynabench/dynasent", "dynabench.dynasent.r1.all",
    cache_dir="../huggingface_cache",
)
r2_dataset = load_dataset(
    "dynabench/dynasent", "dynabench.dynasent.r2.all",
    cache_dir="../huggingface_cache",
)

.json format

In [16]:
r1_train = pd.DataFrame(r1_dataset["train"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)
r2_train = pd.DataFrame(r2_dataset["train"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)
r1_validation = pd.DataFrame(r1_dataset["validation"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)
r2_validation = pd.DataFrame(r2_dataset["validation"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)
r1_test = pd.DataFrame(r1_dataset["test"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)
r2_test = pd.DataFrame(r2_dataset["test"])[['sentence','gold_label']].drop_duplicates().rename(
    columns={'sentence': 'sentence', 'gold_label': 'label'}
)

In [18]:
enc = {
    "positive" : 1, # positive
    "neutral" : 2,  # neutral
    "negative" : 0} # negative
dynasent_train = pd.concat((r1_train, r2_train)).replace({"label": enc}).drop_duplicates()
dynasent_validation = pd.concat((r1_validation, r2_validation)).replace({"label": enc}).drop_duplicates()
dynasent_test = pd.concat((r1_test, r2_test)).replace({"label": enc}).drop_duplicates()

In [22]:
dynasent_train

,sentence,label
0,Roto-Rooter is always good when you need someo...,1
1,It's so worth the price of cox service over he...,1
2,"I placed my order of ""sticky ribs"" as an appet...",2
3,"There is mandatory valet parking, so make sure...",2
4,My wife and I couldn't finish it.,2
...,...,...
13060,This guy was a huge jerk.,0
13061,Never have proper dispensers.,0
13062,The Minister Richard Rosenberg and his wife me...,0
13063,It was a best service.,1


In [24]:
dynasent_train.to_csv('./data/data_raw/DynaSent/train.tsv', sep="\t", index=False)
dynasent_validation.to_csv('./data/data_raw/DynaSent/dev.tsv', sep="\t", index=False)
dynasent_test.to_csv('./data/data_raw/DynaSent/test.tsv', sep="\t", index=False)

Huggingface Datasets format

In [25]:
dynasent_train.index.names = ["idx"]
dynasent_validation.index.names = ["idx"]
dynasent_test.index.names = ["idx"]

In [27]:
dynasent_train_hf = Dataset.from_pandas(dynasent_train)
dynasent_validation_hf = Dataset.from_pandas(dynasent_validation)
dynasent_test_hf = Dataset.from_pandas(dynasent_test)

In [30]:
hf_dataset = DatasetDict()
hf_dataset['train'] = dynasent_train_hf
hf_dataset['validation'] = dynasent_validation_hf
hf_dataset['test'] = dynasent_test_hf

In [31]:
hf_dataset.save_to_disk("./data/data_raw/DynaSent-HF/")